# Домашнее задание к лекции «Продвинутый pandas»

### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd

In [12]:
log = pd.read_csv('data/visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [26]:
def get_source_type(row):
    source = row['traffic_source']
    region = row['region']

    if source == 'yandex' or source == 'google':
        return 'organic'
    if (source == 'paid' or source == 'email') and region == 'Russia':
        return 'ad'
    if (source == 'paid' or source == 'email') and not region == 'Russia':
        return 'other'
    return row['traffic_source']

In [29]:
log['source_type'] = log.apply(get_source_type, axis=1)
log.head()

# Just for test: check rows where traffic_source not yandex and region not Russia
# log.loc[(log.traffic_source != 'yandex') & (log.region != 'Russia')].head(10) 

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [30]:
urls = pd.read_csv('data/URLs.txt')
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [36]:
urls.loc[urls.url.str.contains(r'^.*\/\d{8}\-.*$')].head(10)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


### Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [118]:
ratings = pd.read_csv('data/ratings.csv', usecols=['userId', 'rating', 'timestamp'])
ratings.head()

,userId,rating,timestamp
0,1,2.5,1260759144
1,1,3.0,1260759179
2,1,3.0,1260759182
3,1,2.0,1260759185
4,1,4.0,1260759205


In [134]:
result = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': lambda x: (x.max()-x.min())})
result.loc[result.rating > 100].head()

,rating,timestamp
userId,,
4,204,203560
8,116,85187
15,1700,471393496
17,363,8053
19,423,5282


In [135]:
### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

rzd

In [136]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [137]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [138]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [151]:
df = pd.merge(rzd, auto, how='outer')
df = pd.merge(df, air, how='outer')
df.fillna(0, inplace=True)
df

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [157]:
df.merge(client_base)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
